In [1]:
%load_ext autotime
%load_ext blackcellmagic

time: 87.7 ms (started: 2021-04-19 19:02:30 +03:00)


In [2]:
import re
import string
import numpy as np
import pandas as pd
import polars as pol
from tqdm.auto import tqdm
# from nostril import nonsense
from rich.progress import track
# from derand import Derand

time: 348 ms (started: 2021-04-19 19:02:30 +03:00)


In [3]:
tqdm.pandas()

time: 3.49 ms (started: 2021-04-19 19:02:36 +03:00)


/home/abdullah/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
sentences = pd.read_csv("data/sentences.csv")
sentences_ann = pd.read_csv("data/sentences_annotated.csv")
collections = pd.read_csv("data/collections.csv")
# entries = pd.read_csv("data/entries_raw.csv")

time: 1.82 s (started: 2021-04-19 19:02:50 +03:00)


In [22]:
len(collections)

4924

time: 2.4 ms (started: 2021-04-21 11:12:18 +03:00)


In [5]:
# 1. preprocess
# 2. remove meaningless strings
# 3. match
# 4. detect lang
# 5. split

time: 294 µs (started: 2021-04-19 19:02:54 +03:00)


In [6]:
MIN_CHAR_COUNT = 3
MIN_WORD_COUNT = 3


def filter_trivial(sentence):
    if len(sentence) < MIN_CHAR_COUNT or len(
            sentence.split()) < MIN_WORD_COUNT:
        return ""
    meaningful_words_count = 0
    for word in sentence.split():
        if len(word) > MIN_CHAR_COUNT:
            meaningful_words_count += 1
    if meaningful_words_count < MIN_WORD_COUNT:
        return ""
    return sentence

time: 1.47 ms (started: 2021-04-19 19:02:58 +03:00)


In [7]:
def remove_urls(sentence):
    url_regex = re.compile(
        r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    )
    return url_regex.sub(" ", sentence)

time: 1.07 ms (started: 2021-04-19 19:02:59 +03:00)


In [8]:
# derander = Derand()

time: 600 µs (started: 2021-04-19 19:03:00 +03:00)


In [9]:
def preprocess(sentence, lower=True):
    if not isinstance(sentence, str):
        return ""
    if lower:
        sentence = sentence.lower()
    sentence = re.sub(fr'(\w+)([{string.punctuation}])', r'\1 \2', sentence)
    sentence = re.sub(fr'([{string.punctuation}])(\w+)', r'\1 \2', sentence)
    sentence = re.sub(fr'[\s^]([{string.punctuation}]){2,}[\s$]', r'\1 \2',
                      sentence)
    sentence = remove_urls(sentence)
    sentence = re.sub(r'(.)\1+', r"\1", sentence)
    # sentence = derander.clean(sentence)
    sentence = re.sub("\s+", " ", sentence)
    return sentence.strip()

time: 1.48 ms (started: 2021-04-19 19:03:01 +03:00)


In [10]:
sentences["excerpt_text_pp"] = sentences["excerpt_text"].progress_apply(
    preprocess)


time: 35.2 s (started: 2021-04-19 19:03:05 +03:00)


In [11]:
sentences["excerpt_text_pp"] = sentences["excerpt_text_pp"].progress_apply(
    filter_trivial)


time: 3.12 s (started: 2021-04-19 19:03:40 +03:00)


In [12]:
print(sentences.shape[0])
before = sentences.shape[0]
sentences = sentences[~sentences["excerpt_text_pp"].eq("")]
print(sentences.shape[0], before - sentences.shape[0])

523862
418968 104894
time: 169 ms (started: 2021-04-19 19:03:43 +03:00)


In [13]:
print(sentences.shape[0])
before = sentences.shape[0]
sentences = sentences[~sentences["excerpt_text_pp"].duplicated()]
print(sentences.shape[0], before - sentences.shape[0])

418968
355715 63253
time: 171 ms (started: 2021-04-19 19:03:43 +03:00)


In [14]:
sentences["unique_id"] = sentences["lead_id"].astype(str) + "_" + sentences["excerpt_id"].astype(str)
sentences_ann["unique_id"] = sentences_ann["lead_id"].astype(str) + "_" + sentences_ann["excerpt_id"].astype(str)

time: 1.18 s (started: 2021-04-19 19:03:59 +03:00)


In [15]:
print(sentences_ann.shape[0])
before = sentences_ann.shape[0]
sentences_ann = sentences_ann[sentences_ann["unique_id"].isin(sentences["unique_id"])]
print(sentences_ann.shape[0], before - sentences_ann.shape[0])

523862
355715 168147
time: 278 ms (started: 2021-04-19 19:04:02 +03:00)


In [16]:
print(collections.shape[0])
before = collections.shape[0]
collections = collections[collections["lead_id"].isin(sentences["lead_id"])]
print(collections.shape[0], before - collections.shape[0])

5291
4924 367
time: 11.6 ms (started: 2021-04-19 19:04:07 +03:00)


In [17]:
sentences.to_csv("data_pp/sentences.csv", index=None)
sentences_ann.to_csv("data_pp/sentences_annotated.csv", index=None)
collections.to_csv("data_pp/collections.csv", index=None)

time: 5.51 s (started: 2021-04-19 19:04:17 +03:00)


In [21]:
with open("data_pp.txt", "w") as f:
    for s in sentences["excerpt_text"]:
        f.write(f"{s}\n")

time: 195 ms (started: 2021-04-19 19:15:05 +03:00)
